In [9]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

import feather
import warnings
import time
import sys
import datetime
import re
import gc
import os

from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

from scipy.stats import mode
from scipy import stats
from sklearn import preprocessing


from IPython.core.display import display, HTML

In [2]:
historical_transactions = pd.read_csv('C:\\kdata\\historical_transactions.csv', parse_dates=['purchase_date'])

In [13]:
df_train = pd.read_csv('data\\train.csv')

In [3]:
def featurize_train_test(df):
    # to datetime
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])

    # datetime features
    df['quarter'] = df['first_active_month'].dt.quarter
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days

    df['days_feature1'] = df['elapsed_time'] * df['feature_1']
    df['days_feature2'] = df['elapsed_time'] * df['feature_2']
    df['days_feature3'] = df['elapsed_time'] * df['feature_3']

    df['days_feature1_ratio'] = df['feature_1'] / df['elapsed_time']
    df['days_feature2_ratio'] = df['feature_2'] / df['elapsed_time']
    df['days_feature3_ratio'] = df['feature_3'] / df['elapsed_time']


    df['feature_sum'] = df['feature_1'] + df['feature_2'] + df['feature_3']
    df['feature_mean'] = df['feature_sum']/3
    df['feature_max'] = df[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
    df['feature_min'] = df[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
    df['feature_var'] = df[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

    t1 = pd.get_dummies(df_train.feature_1, prefix = 'feature1')
    t2 = pd.get_dummies(df_train.feature_2, prefix = 'feature2')
    t3 = pd.get_dummies(df_train.feature_3, prefix = 'feature3')

    df[t1.columns] = t1
    df[t2.columns] = t2
    df[t3.columns] = t3

    del t1, t2, t3
    gc.collect()

    return df

In [14]:
train_fe = featurize_train_test(train)

In [15]:
train_fe

first_active_month          card_id  feature_1  feature_2  feature_3  \
0              2017-06-01  C_ID_92a2005557          5          2          1   
1              2017-01-01  C_ID_3d0044924f          4          1          0   
2              2016-08-01  C_ID_d639edf6cd          2          2          0   
3              2017-09-01  C_ID_186d6a6901          4          3          0   
4              2017-11-01  C_ID_cdbd2c0db2          1          3          0   
5              2016-09-01  C_ID_0894217f2f          4          2          0   
6              2016-12-01  C_ID_7e63323c00          3          2          1   
7              2017-09-01  C_ID_dfa21fc124          3          2          1   
8              2017-08-01  C_ID_fe0fdac8ea          2          1          0   
9              2016-08-01  C_ID_bf62c0b49d          2          2          0   
10             2016-10-01  C_ID_92853cdb2c          5          2          1   
11             2016-03-01  C_ID_269d816788          2          2          0   
12             2017-09-01  C_ID_61d50d7057          5          2          1   
13             2017-04-01  C_ID_4e07413433          2          1          0   
14             2017-08-01  C_ID_b6302b31c6          3          2          1   
15             2017-02-01  C_ID_3b69154173          2          1          0   
16             2016-07-01  C_ID_9feec11e78          2          2          0   
17             2016-12-01  C_ID_f6658dbefe          4          2          0   
18             2017-11-01  C_ID_4a7dda0f9e          2          1          0   
19             2017-01-01  C_ID_6adae2a906          4          1          0   
20             2017-09-01  C_ID_0b70ca7347          5          2          1   
21             2016-06-01  C_ID_25b2509282          1          2          0   
22             2017-09-01  C_ID_82fb11db22          2          2          0   
23             2017-10-01  C_ID_e03db05bde          2          2          0   
24             2015-10-01  C_ID_dcb7c76747          5          1          1   
25             2016-04-01  C_ID_749cfc0c3a          2          2          0   
26             2017-07-01  C_ID_4a97126360          2          3          0   
27             2017-09-01  C_ID_17dea8d446          3          3          1   
28             2016-01-01  C_ID_fcef0dfd67          5          1          1   
29             2016-06-01  C_ID_82868156f0          5          2          1   
...                   ...              ...        ...        ...        ...   
201887         2017-01-01  C_ID_fa343b1b60          2          2          0   
201888         2016-08-01  C_ID_8fb51d131f          4          2          0   
201889         2017-07-01  C_ID_ea7df02b8d          3          2          1   
201890         2015-09-01  C_ID_82554b8b1f          2          2          0   
201891         2016-10-01  C_ID_ec1fdc9c13          5          2          1   
201892         2017-04-01  C_ID_be5a444ff5          4          3          0   
201893         2017-10-01  C_ID_6c40a68e73          5          1          1   
201894         2015-07-01  C_ID_d02dd24e29          2          2          0   
201895         2017-01-01  C_ID_2e16d5537d          5          1          1   
201896         2015-03-01  C_ID_808ba69dff          3          3          1   
201897         2016-11-01  C_ID_095bca2d92          5          2          1   
201898         2017-06-01  C_ID_123b4b8d1e          1          1          0   
201899         2017-10-01  C_ID_58e359763e          5          1          1   
201900         2017-07-01  C_ID_0032aebb26          3          2          1   
201901         2017-11-01  C_ID_3814ea6382          3          2          1   
201902         2017-01-01  C_ID_b9cd68366b          1          3          0   
201903         2016-05-01  C_ID_b860347cb0          3          2          1   
201904         2017-01-01  C_ID_debbad9dec          3          3          1   
201905         2017-06-01  C_ID_75a2251acf          3    

In [17]:
df_category3_count = historical_transactions.groupby('card_id')['category_3'].value_counts()

In [20]:
df_category3_count = df_category3_count.unstack()

In [21]:
df_category3_fraction = np.divide(df_category3_count, df_category3_count.sum(axis = 1).values.reshape(-1,1))

In [23]:
city_id_count = historical_transactions.groupby('city_id')['city_id'].count()

In [26]:
np.log(city_id_count).hist()

In [38]:
bins = pd.qcut(np.log(city_id_count), 5, duplicates='drop')

In [54]:
pd.qcut(np.log(city_id_count), 5, duplicates='drop',labels=range(5))

city_id
-1      4
 1      4
 2      1
 3      3
 4      4
 6      2
 7      2
 8      4
 9      2
 10     2
 11     4
 12     4
 13     2
 14     3
 16     0
 17     4
 18     0
 19     4
 20     4
 21     4
 22     2
 23     2
 24     4
 25     4
 26     0
 28     2
 29     2
 30     4
 32     0
 33     3
       ..
 314    4
 315    0
 316    0
 318    2
 319    0
 320    3
 321    0
 322    4
 323    3
 325    2
 326    0
 327    0
 328    2
 329    2
 330    2
 331    4
 333    4
 334    3
 335    0
 336    0
 337    1
 338    1
 339    3
 340    4
 341    2
 342    1
 343    0
 344    4
 345    0
 347    3
Name: city_id, Length: 308, dtype: category
Categories (5, int64): [0 < 1 < 2 < 3 < 4]

In [40]:
df1 = historical_transactions['city_id'].map(bins)

In [51]:
df1

0           (11.183, 15.391]
1           (11.183, 15.391]
2           (11.183, 15.391]
3           (11.183, 15.391]
4           (11.183, 15.391]
5           (11.183, 15.391]
6           (11.183, 15.391]
7           (10.474, 11.183]
8           (11.183, 15.391]
9           (11.183, 15.391]
10          (11.183, 15.391]
11          (11.183, 15.391]
12          (11.183, 15.391]
13          (11.183, 15.391]
14          (11.183, 15.391]
15          (11.183, 15.391]
16          (11.183, 15.391]
17          (11.183, 15.391]
18          (11.183, 15.391]
19          (11.183, 15.391]
20          (11.183, 15.391]
21          (11.183, 15.391]
22          (11.183, 15.391]
23          (11.183, 15.391]
24          (10.474, 11.183]
25          (11.183, 15.391]
26          (11.183, 15.391]
27          (11.183, 15.391]
28          (11.183, 15.391]
29          (11.183, 15.391]
                  ...       
29112331    (11.183, 15.391]
29112332    (11.183, 15.391]
29112333    (11.183, 15.391]
29112334    (1